In [2]:
import json
import os
import polars as pl
from polars import col as c
import polars.selectors as cs
import re
from data_federation.input_model import SmallflexInputSchema
import owncloud
import streamlit as st
from datetime import date
import plotly.graph_objs as go
from data_federation.parser.market_price import parse_market_price, parse_apg_market_price
from data_federation.parser.market_price import parse_da_ida_market_price, parse_reg_frr_market_price, parse_reg_frc_market_price, parse_rte_cap_market_price, parse_rte_ene_market_price, parse_swissgrid_cap_market_price
from typing_extensions import Literal

from data_federation.parser.discharge_flow import parse_discharge_flow_forecast, parse_discharge_flow_historical
from data_federation.parser.weather import parse_weather_historical, parse_weather_forecast
from data_federation.parser.aegina_hydro_power_plant import parse_aegina_hydro_power_plant
from data_federation.parser.merezenbach_hydro_power_plant import parse_merezenbach_hydro_power_plant
from data_federation.parser.morel_hydro_power_plant import parse_morel_hydro_power_plant
from data_federation.parser.aegina_wind_power_plant import parse_aegina_wind_power_plant
from data_federation.parser.parser_pipeline import input_pipeline
from general_function import dictionary_key_filtering, scan_switch_directory, modify_string, generate_uuid, pl_to_dict
from polars_function import modify_string_col, generate_uuid_col
from data_display.input_data_plots import plot_market_prices
from data_federation.parser.hydro_power_plant import get_hydro_power_plant_data
from config import settings
import plotly.express as px
from datetime import timedelta, date
from utility.pyomo_preprocessing import *
from datetime import timedelta, date, datetime, timezone
from datetime import timedelta, date
import numpy as np
from typing_extensions import Optional
import pyomo.environ as pyo
from itertools import product
import tarfile
from pyomo_models.baseline.baseline_input import BaseLineInput
from pyomo_models.baseline.first_stage.first_stage_pipeline import BaselineFirstStage
from pyomo_models.baseline.second_stage.second_stage_pipeline import BaselineSecondStage
from data_display.baseline_plots import plot_first_stage_result, plot_second_stage_result
import owncloud
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro
from pathlib import Path
from datetime import datetime, timedelta
from plotly.subplots import make_subplots

import tqdm
from general_function import  generate_log, extract_archive, scan_folder, build_non_existing_dirs
from polars_function import concat_list_of_list
from data_federation.input_model import SmallflexInputSchema
from data_display.input_data_plots import plot_forecast
import numpy as np
import plotly
from config import settings

os.chdir(os.getcwd().replace("/src", ""))

log = generate_log(name="file_")
input_file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES)) # type: ignore
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES)) # type: ignore

In [ ]:
col_to_drop =  ["time", "weekday", "yy", "dd", "mm", "hh"]
file_type = input_file_names["wsl_discharge_flow_forecast"]
folder_name = input_file_names["wsl_forecast_data"]
forecast_data: pl.DataFrame = pl.DataFrame()
measurement_data: pl.DataFrame = pl.DataFrame()
extract_archive(file_name=input_file_names["wsl_forecast_data"])

file_names_list = scan_folder(folder_name=os.path.splitext(folder_name)[0], extension=".rda", file_names=file_type)
file_names_list = sorted(file_names_list)
test = 30
n_rows = len(file_names_list) - 7
plot_folder = output_file_names["input_data_plot"]

fig = go.Figure()
COLORS = px.colors.qualitative.Plotly
fig = make_subplots(
            rows=n_rows, cols = 1
        )
for i in range(n_rows):
    data: pl.DataFrame = pl.DataFrame()
    for j, file_name in enumerate(file_names_list[i:i+7]):
        if j==0:
            actual_date = datetime.strptime(Path(file_name).parent.name  , '%Y-%m-%d')
        names = ro.r['load'](file_name) # type: ignore
        all_df = ro.r[names[0]]
        
        data: pl.DataFrame = pl.from_pandas(pandas2ri.rpy2py(all_df.rx2(all_df.names[0]))) # type: ignore
        data = data.select(
                pl.from_epoch(c("time")).alias("time"),
                c("Gle100_5") ,
            ).filter(c("time") < actual_date)
        
        fig.add_trace(
            go.Scatter(
                x=data["time"], y=data["Gle100_5"], 
                mode='lines', name=Path(file_name).parent.name, line=dict(color=COLORS[j]),
                legendgroup=f"day{i}"),
            row=i+1, col=1)

    fig.update_traces(selector=dict(legendgroup=f"day{i}"), legendgrouptitle_text=actual_date.strftime("%Y-%m-%d"))
        
fig.update_layout(
            margin=dict(t=60, l=65, r= 10, b=60), 
            width=1000,   # Set the width of the figure
            height=n_rows*300,
            legend_tracegroupgap=146
        )

fig.write_html(f"{plot_folder}/discharge_flow_measurement_differences.html")

In [ ]:
col_to_drop =  ["time", "weekday", "yy", "dd", "mm", "hh"]
file_type = input_file_names["wsl_weather_forecast"]
folder_name = input_file_names["wsl_forecast_data"]
forecast_data: pl.DataFrame = pl.DataFrame()
measurement_data: pl.DataFrame = pl.DataFrame()

file_names_list = scan_folder(folder_name=os.path.splitext(folder_name)[0], extension=".rda", file_names=file_type)
file_names_list = sorted(file_names_list)
test = 30
n_rows = len(file_names_list) - 6
n_rows = 2
plot_folder = output_file_names["input_data_plot"]

COLORS = px.colors.qualitative.Plotly
fig = make_subplots(
            rows=n_rows, cols = 1
        )


col_name = "windGri200_41"
for i in range(n_rows):
    data: pl.DataFrame = pl.DataFrame()
    for j, file_name in enumerate(file_names_list[i:i+6]):
        if j==0:
            actual_date = datetime.strptime(Path(file_name).parent.name  , '%Y-%m-%d')
        print(actual_date)
        names = ro.r['load'](file_name) # type: ignore
        all_df = ro.r[names[0]]
        
        data: pl.DataFrame = pl.from_pandas(pandas2ri.rpy2py(all_df.rx2(all_df.names[0]))) # type: ignore
        data = data.select(
                pl.from_epoch(c("time")).alias("time"),
                c(col_name) ,
            ).filter(c("time") < actual_date)
        
        fig.add_trace(
            go.Scatter(
                x=data["time"], y=data[col_name], 
                mode='lines', name=Path(file_name).parent.name, line=dict(color=COLORS[j]),
                legendgroup=f"day{i}"),
            row=i+1, col=1)

    fig.update_traces(selector=dict(legendgroup=f"day{i}"), legendgrouptitle_text=actual_date.strftime("%Y-%m-%d"))
        
fig.update_layout(
            margin=dict(t=60, l=65, r= 10, b=60), 
            width=1000,   # Set the width of the figure
            height=n_rows*300,
            legend_tracegroupgap=146
        )

fig.write_html(f"{plot_folder}/wind_measurement_differences.html")

2024-06-21 00:00:00
2024-06-21 00:00:00
2024-06-21 00:00:00
2024-06-21 00:00:00
2024-06-21 00:00:00
2024-06-21 00:00:00
2024-06-22 00:00:00
2024-06-22 00:00:00
2024-06-22 00:00:00
2024-06-22 00:00:00
2024-06-22 00:00:00
2024-06-22 00:00:00


In [14]:
data

time,windGri200_41
datetime[μs],f64
2024-06-10 00:00:00,2.42
2024-06-10 01:00:00,1.79
2024-06-10 02:00:00,1.76
2024-06-10 03:00:00,2.46
2024-06-10 04:00:00,2.96
…,…
2024-06-21 19:00:00,3.67
2024-06-21 20:00:00,3.33
2024-06-21 21:00:00,2.89


In [9]:
names = ro.r['load'](file_names_list[0])
all_df = ro.r[names[0]]
data: pl.DataFrame = pl.from_pandas(pandas2ri.rpy2py(all_df.rx2(all_df.names[0]))) 

In [12]:
file_names_list

['.cache/input/gries_wsl_input_data/prev_SF/2024-06-21/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-22/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-23/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-24/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-25/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-26/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-27/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-28/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-29/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-06-30/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-07-01/RRouting_Gen_icon2_meteo.rda',
 '.cache/input/gries_wsl_input_data/prev_SF/2024-07-02

In [ ]:
data

In [ ]:
small_flex_input_schema = SmallflexInputSchema()

small_flex_input_schema = small_flex_input_schema.duckdb_to_schema(output_file_names["duckdb_input"])

kwargs: dict = {
        "small_flex_input_schema": small_flex_input_schema, 
        "input_file_names": input_file_names
    }

kwargs["small_flex_input_schema"] = parse_weather_forecast(**kwargs)

small_flex_input_schema: SmallflexInputSchema = kwargs["small_flex_input_schema"]
small_flex_input_schema.schema_to_duckdb(output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.82it/s]
Parsing weather forecast files:   4%|▍         | 5/127 [00:33<13:18,  6.54s/it]